In [1]:
from __future__ import print_function, division, absolute_import
import os
import sys
sys.path.insert(0, "C:\\Users\\Matt\\Desktop\\orbithunter\\")
from orbithunter import *
import numpy as np

In [2]:
orbit1 = read_h5('OrbitKS_L31p245_T46p754.h5')
orbit2 = read_h5('OrbitKS_L26p931_T41p266.h5')
orbit3 = read_h5('OrbitKS_L26p427_T80p246.h5')
orbit4 = read_h5('OrbitKS_L27p038_T23p329.h5')

In [3]:
orbit1 = rediscretize(orbit1, new_shape=(64, 64))

orbit2 = rediscretize(orbit2, new_shape=(64, 64))

orbit3 = rediscretize(orbit3, new_shape=(64, 64))

orbit4 = rediscretize(orbit4, new_shape=(64, 64))

In [4]:
from orbithunter.gluing import tile_dictionary_ks

In [5]:
OrbitKS.dimensions()

['T', 'L']

In [6]:
tile_shape = (64, 64)
tiled_orbit = tile(np.array([[0,1],[1,0]]), tile_shape, tile_dictionary_ks(padded=True), OrbitKS)

TypeError: zip argument #1 must support iteration

In [11]:
iterable_of_orbits = [OrbitKS(T=0.5, L=22), orbit2, orbit3, orbit4]
axis = 0 
# need to iterate over all orbits, find the total dimension and # of points, then create the new_shape tuples.
total_dimension = 0
total_discretization = 0
for o in iterable_of_orbits:
    total_dimension += o.parameters[o.dimensions()[axis]]
    total_discretization += o.parameters['field_shape'][axis]

# Replace the # of points along axis with the rescaled values based upon the total extent of the axis dimension.

new_shapes = [tuple(int(2 * ((((total_discretization * o.parameters[o.dimensions()[axis]]) / total_dimension)+1) // 2))
              if i == axis else o.parameters['field_shape'][i] for i in range(len(o.shape)))
              for o in iterable_of_orbits]
new_shapes

[(0, 32), (64, 64), (124, 64), (36, 64)]

In [8]:
iterable_of_reshaped_orbits = [rediscretize(o, new_shape=shp)
                               for o, shp in zip(iterable_of_orbits, new_shapes)]

In [30]:
total_discretization = 202
total_dimension=179


In [31]:
np.mod([max([2, int(2 * ((((total_discretization * test_dimension) / total_dimension) + 1) // 2))]) for test_dimension in np.arange(1, 200, 0.01)], 2).sum()


0

In [22]:
 int(2 * ((((total_discretization * test_dimension) / total_dimension) + 1) // 2))

58

In [ ]:
converge(tiled_orbit, verbose=True)

In [ ]:
pcc, persistence = orbit_persistence(orbit1)

In [ ]:
gudhi_plot(persistence)